<font size = "5"> **EELS_Tools:  [Spectroscopy](../4_EELS_Tools.ipynb)** </font>

<hr style="height:1px;border-top:4px solid #FF8200" />

# Analysis of Core-Loss EELS Spectra
<br>

[<img src=https://www.coeuscreativegroup.com/wp-content/uploads/2020/04/download-button.png, width=125>](https://raw.githubusercontent.com/pycroscopy/pyTEMlib/main/notebooks/EELS/Analysis_Core_Loss.ipynb) 

[![OpenInColab](https://colab.research.google.com/assets/colab-badge.svg)](
    https://colab.research.google.com/github/pycroscopy/pyTEMlib/blob/main/notebooks/EELS/Analysis_Core_Loss.ipynb)
    
part of 

<font size = "5"> **[pyTEMlib](https://pycroscopy.github.io/pyTEMlib/about.html)**</font>

a [pycroscopy](https://pycroscopy.github.io/pycroscopy/about.html) ecosystem package


Notebook by Gerd Duscher, 2023

Microscopy Facilities<br>
Institute of Advanced Materials & Manufacturing<br>
The University of Tennessee, Knoxville

Model based analysis and quantification of data acquired with transmission electron microscopes




## Content

Quantitative determination of chemical composition from a core-loss EELS spectrum

Relavant notebooks in course [MSE 672: Introduction to TEM]()  give an in depth explanation of the method.

Please cite:

[M. Tian et  al. *Measuring the areal density of nanomaterials by electron energy-loss spectroscopy*
Ultramicroscopy Volume 196, 2019, pages 154-160](https://doi.org/10.1016/j.ultramic.2018.10.009)

as a reference of this quantification method.

## Prerequesites
This notebook will only run with  ``pyTEMlib version 0.2023.5.0`` or higher.

### Install missing packages
This can take a while for the first time. This code will have to run at least once on your computer and for each colab session.

In [2]:
import sys

from pkg_resources import get_distribution, DistributionNotFound

def test_package(package_name):
    """Test if package exists and returns version or -1"""
    try:
        version = (get_distribution(package_name).version)
    except (DistributionNotFound, ImportError) as err:
        version = '-1'
    return version


# pyTEMlib setup ------------------
if test_package('pyTEMlib') < '0.2023.8.1':
    print('installing pyTEMlib')
    !{sys.executable} -m pip install git+https://github.com/pycroscopy/sidpy
    !{sys.executable} -m pip install git+https://github.com/pycroscopy/SciFiReaders.git@main -q
    !{sys.executable} -m pip install git+https://github.com/pycroscopy/pyTEMlib.git@main -q --upgrade
    
if 'google.colab' in sys.modules:
    !{sys.executable} -m pip install numpy==1.25.4
# ------------------------------
print('done')

installing pyTEMlib
  Cloning https://github.com/pycroscopy/sidpy to c:\users\gduscher\appdata\local\temp\pip-req-build-t5foc5pm
  Resolved https://github.com/pycroscopy/sidpy to commit 78a293b0e2f2d3f6aa2773a484918c0c2ba94ebf
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/pycroscopy/sidpy 'C:\Users\gduscher\AppData\Local\Temp\pip-req-build-t5foc5pm'


done


Restart the Runtime in Runtime Menu above or pres "Ctrl-M"

### Load the relevant packages

In [1]:
%matplotlib ipympl
import numpy as np
import matplotlib.pylab as plt 
import sys

if 'google.colab' in sys.modules:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive

import sys
sys.path.insert(0, '../..')
%load_ext autoreload
%autoreload 2

import pyTEMlib
import pyTEMlib.file_tools as ft     # File input/ output library
import pyTEMlib.image_tools as it
import pyTEMlib.eels_tools  as eels        # EELS methods 

import pyTEMlib.interactive_eels as ieels  # Dialogs for EELS input and quantification

# For archiving reasons it is a good idea to print the version numbers out at this point
print('pyTEMlib version: ', pyTEMlib.__version__)

__notebook__ = 'analyze_core_loss'
__notebook_version__ = '2023_05_21'

if 'google.colab' in sys.modules:
    drive.mount("/content/drive")

You don't have igor2 installed.     If you wish to open igor files, you will need to install it     (pip install igor2) before attempting.
You don't have gwyfile installed.     If you wish to open .gwy files, you will need to      install it (pip install gwyfile) before attempting.
Symmetry functions of spglib enabled
pyTEMlib version:  0.2023.8.1


## Load and plot a spectrum

As an example we load the spectrum **1EELS Acquire (high-loss).dm3** from the *example data* folder.

Please see [Loading an EELS Spectrum](LoadEELS.ipynb) for details on storage and plotting.

First a dialog to select a file will apear.

Then the spectrum plot and ``Spectrum Info`` dialog will appear, in which we set the experimental parameters.

Please use the ``Set Energy Scale`` button to change the energy scale. When pressed a new dialog and a cursor will appear in which one is able to set the energy scale based on known features in the spectrum.


In [2]:
# -----Input -------#
load_example = False

if load_example:
    datasets = ft.open_file('../../example_data/EELS_STO3.hf5')
else:  
    fileWidget = ft.FileWidget()

In [54]:
if not load_example:
    datasets = fileWidget.datasets
infoWidget= ieels.InfoWidget(datasets)


AppLayout(children=(GridspecLayout(children=(Dropdown(description='Main Dataset:', layout=Layout(grid_area='wi…

## Chemical Composition 
The fit of the cross-section and background to the spectrum results in the chemical composition. If the calibration is correct this composition is given as areal density in atoms/nm$^2$




In [43]:
print('Auto Quantification')
eels.auto_chemical_composition(infoWidget.dataset)

Auto Quantification

Relative composition: 
Ti: 72.2%  Sb: 24.3%  Te: 3.5%  


###  Fit of Data
A dashboardw will open.
Press **Elements** to select the elements first.

Pres``Quantification`` button. Adjust parameters as needed and check fit by pressing the ``Quantification`` button again.

Select the ``Fit Area`` button will show you which parts of the spectrum you choose to fit.

Changing the multiplier value will make a simulation of your spectrum.

Probability will change to atoms/nm$^2$ if you selected a reference for the counts in the info dashboard above.
Edges button will display the edges of the te selected elements.

If a low-loss spectrum is selected a 

In [87]:
datasets['Channel_000'].metadata['edges']={}
compositionWidget = ieels.CompositionWidget(datasets)

AppLayout(children=(GridspecLayout(children=(ToggleButton(value=False, button_style='info', description='Fit A…

### Output of Results

In [90]:
edges = compositionWidget.dataset.metadata['edges']
element = []
areal_density = []
for key, edge in edges.items():
    if key.isdigit():
        element.append(edge['element'])
        areal_density.append(edge['areal_density'])

print('Relative chemical composition of ', compositionWidget.dataset.title)
for i in range(len(element)):
    print(f'{element[i]}: {areal_density[i]/np.sum(areal_density)*100:.1f} %')
    
saved_edges_metadata = edges

Relative chemical composition of  EELS_STO2
Ti: 20.9 %
O: 79.1 %


### Log Data
All the data associated with quantification are stored in the ``metadata`` attribute. The relevant keys of that dictionary are:  ``experiment`` and  ``edges``.

In [44]:
compositionWidget.dataset.view_metadata()

experiment :
	single_exposure_time : 2.0
	exposure_time : 20.0
	number_of_frames : 10
	collection_angle : 33.0
	convergence_angle : 30.0
	acceleration_voltage : 200000.0
	flux_ppm : 0
	count_conversion : 1
	beam_current : 0
filename : c:\Users\gduscher\Documents\Github\pyTEMlib\example_data\EELS_STO2.dm3
edges :
	0 :
		z : 22
		symmetry : L3
		element : Ti
		onset : 455.5
		end_exclude : 505.5
		start_exclude : 450.5
		all_edges :
			L3 :
				onset : 455.5
			L2 :
				onset : 461.5
			L1 :
				onset : 563.6999999999999
		chemical_shift : 0.0
		areal_density : 7065319769996.836
		original_onset : 455.5
		data : [1.23264220e-09 1.22917273e-09 1.22570326e-09 ... 4.28971162e-10
 4.28439070e-10 4.27906979e-10]
		X_section_type : XRPA
		X_section_source : pyTEMlib
	model :
		background : energy_loss:  energy-loss (eV) of size (2048,)
		background-poly_0 : -18905.269730586166
		background-poly_1 : -5.10291026745673
		background-poly_2 : 0.005046721263305648
		background-A : 3837665365.6060038

## ELNES
The electron energy-loss near edge structure is determined by fititng the spectrum after quantification model subtraction. 

First smooth the spectrum (only 0 iteration available at this time) and then 
find the number of peaks you want (Can be repeated as oftern as one wants).

Please note that the peaks do not necessarily have a physcial meaning as there are peaks with 
positive and negative amplitudes. The goal is to get a smooth representation of the spectrum.
A difference of a good fit and the data should result in a residual of noise only. To achieve such a good fit, the careful setting of the edge onset is important.


In [92]:
datasets['Channel_000'].metadata['peak_fit'] ={}
peakFitWidget = ieels.PeakFitWidget(datasets)

AppLayout(children=(GridspecLayout(children=(Button(description='Fit Area', layout=Layout(grid_area='widget001…

### Output

In [48]:
areas = []
for p, peak in peakFitWidget.peaks['peaks'].items():
    area = np.sqrt(2* np.pi)* peak['amplitude'] * np.abs(peak['width'] / np.sqrt(2 *np.log(2))) 
    areas.append(area)
    if 'associated_edge' not in peak:
        peak['associated_edge']= ''
    print(f"peak  {p}: position: {peak['position']:7.1f}, area: {area:12.3f} associated edge: {peak['associated_edge']}")
#print(f'\n M4/M5 peak 2 to peak 1 ratio: {(areas[1])/areas[0]:.2f}')

peak  0: position:   540.6, area: -2171440.358 associated edge: O-K1
peak  1: position:   540.6, area:  2044736.464 associated edge: O-K1
peak  2: position:   462.3, area:  1019741.883 associated edge: Ti-L2
peak  3: position:   475.4, area:  -421663.981 associated edge: Ti-L2
peak  4: position:   457.3, area:   364278.003 associated edge: Ti-L3
peak  5: position:   454.9, area:   266951.091 associated edge: 
peak  6: position:   559.7, area:  -229694.131 associated edge: O-K1
peak  7: position:   356.7, area:   147740.995 associated edge: 


### Log Data
All the data associated with quantification are stored in the ``metadata`` attribute. 
The new key in that dictionary is: ``peak_fit``.

In [49]:
infoWidget.dataset.metadata.keys()

dict_keys(['experiment', 'filename', 'edges', 'peak_fit'])

## Save and Close File
The dataset with the metadata should be saved for further quantification. 
The file needs to be closed to be used with other notebooks.

In [50]:
h5_group = ft.save_dataset(infoWidget.datasets, '../../example_data/EELS_STO3.hf5')
h5_group.file.close()

c:\Users\gduscher\AppData\Local\anaconda3\envs\pyTEMlib\Lib\site-packages\pyNSID\io\hdf_utils.py:376: FutureWarning: validate_h5_dimension may be removed in a future version
  warn('validate_h5_dimension may be removed in a future version',
